In [1]:
# from desktop w/ gpu
import torch
import numpy as np
import cv2
import time
import os
from enum import Enum
from multipledispatch import dispatch
import mrs3 as mr
import interpolation as inter

%load_ext autoreload
%autoreload 2

lenna_path = 'Lenna_(test_image).png'

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(device = 0))
print(torch.cuda.device_count())
print(torch.backends.cudnn.version())



True
NVIDIA GeForce RTX 3060
1
91000


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import time

In [ ]:

# Chrome 옵션 설정
chrome_options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": os.path.abspath("downloads"),  # 다운로드 경로 설정
    "download.prompt_for_download": False,                       # 다운로드 확인창 비활성화
    "download.directory_upgrade": True
}
chrome_options.add_experimental_option("prefs", prefs)

# 웹드라이버 초기화
driver = webdriver.Chrome(options=chrome_options)
driver.get("http://imgtest.com")

# 다운로드 폴더 생성
os.makedirs("downloads", exist_ok=True)

# 사용자 동의 체크박스 처리
try:
    agreement_checkbox = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "agree-checkbox"))
    )
    agreement_checkbox.click()
except:
    print("동의 체크박스를 찾을 수 없습니다")

# 이미지 업로드 폴더 경로
upload_folder = "upload_images"
image_files = sorted([f for f in os.listdir(upload_folder) if f.endswith(('.png', '.jpg', '.jpeg'))])

for idx, image in enumerate(image_files):
    # 이미지 업로드
    file_path = os.path.abspath(os.path.join(upload_folder, image))
    
    browse_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Browse')]"))
    )
    browse_button.click()
    
    # 파일 입력 요소 찾기
    file_input = driver.find_element(By.XPATH, "//input[@type='file']")
    driver.execute_script("arguments[0].style.display = 'block';", file_input)
    file_input.send_keys(file_path)
    
    # 업로드 버튼 클릭
    upload_btn = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Upload')]"))
    )
    upload_btn.click()
    
    # 처리 버튼 클릭
    process_btn = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'right-arrow')]"))
    )
    process_btn.click()
    
    # 처리 완료 대기 (다운로드 버튼 활성화 확인)
    WebDriverWait(driver, 30).until(
        EC.visibility_of_element_located((By.XPATH, "//a[contains(text(), 'Download')]"))
    )
    
    # 다운로드 버튼 클릭
    download_btn = driver.find_element(By.XPATH, "//a[contains(text(), 'Download')]")
    download_btn.click()
    
    # 다음 이미지 처리 전 대기
    time.sleep(2)

driver.quit()


In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os

# Chrome 옵션 설정
chrome_options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": os.path.join(os.getcwd(), "Downloads"),  # 다운로드 경로 설정
    "download.prompt_for_download": False,  # 다운로드 확인창 비활성화
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
}
chrome_options.add_experimental_option("prefs", prefs)

# WebDriver 초기화
driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 20)

try:
    # 이미지 업로드 페이지 접속
    driver.get("https://imgpresso.co.kr")
    
    # 파일 업로드 처리
    upload_input = wait.until(EC.presence_of_element_located(
        (By.XPATH, "//input[@type='file']")))
    upload_input.send_keys("/home/primarina314/Projects/MRS3/Lenna_(test_image).png")  # 업로드할 이미지 경로 지정
    print('test1')
    try:
        option_dropdown = wait.until(EC.presence_of_element_located(
            (By.XPATH, "//select[@id='i3eeph']")))
        Select(option_dropdown).select_by_visible_text("Very")
        print("옵션 선택 완료")
    except:
        print("옵션 선택 요소를 찾을 수 없음")
        raise

    # 4. 확인 버튼 클릭
    confirm_btn = wait.until(EC.element_to_be_clickable(
        (By.XPATH, "//button[contains(text(), '용량 줄이기기')]")))
    confirm_btn.click()
    print("확인 버튼 클릭 완료")
    time.sleep(2)  # 옵션 적용 대기

    # 처리 시작 버튼 클릭 (사이트 구조에 따라 선택자 수정 필요)
    process_btn = wait.until(EC.element_to_be_clickable(
        (By.XPATH, "//button[contains(text(), '처리 시작')]")))
    process_btn.click()
    print('test2')
    # 처리 완료 대기 (사이트 구조에 따라 수정 필요)
    wait.until(EC.visibility_of_element_located(
        (By.XPATH, "//div[contains(@class, 'processing-complete')]")))
    print('test3')
    # 다운로드 버튼 클릭
    download_btn = wait.until(EC.element_to_be_clickable(
        (By.XPATH, "//a[contains(text(), '다운로드')]")))
    download_btn.click()
    print('test4')
    # 파일 다운로드 대기
    time.sleep(5)  # 다운로드 완료까지 충분한 시간 확보

finally:
    driver.quit()


test1
옵션 선택 요소를 찾을 수 없음


InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=137.0.7151.55)
Stacktrace:
#0 0x55e51b44d29a <unknown>
#1 0x55e51aeb9040 <unknown>
#2 0x55e51ae9f87e <unknown>
#3 0x55e51aec7bb4 <unknown>
#4 0x55e51af3911f <unknown>
#5 0x55e51af56472 <unknown>
#6 0x55e51af30463 <unknown>
#7 0x55e51aefcc55 <unknown>
#8 0x55e51aefd8c1 <unknown>
#9 0x55e51b4121bb <unknown>
#10 0x55e51b4160c9 <unknown>
#11 0x55e51b3faa39 <unknown>
#12 0x55e51b416c78 <unknown>
#13 0x55e51b3df6bf <unknown>
#14 0x55e51b43a948 <unknown>
#15 0x55e51b43ab26 <unknown>
#16 0x55e51b44c176 <unknown>
#17 0x7f82a20f2aa4 <unknown>
#18 0x7f82a217fc3c <unknown>


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def automate_image_processing():
    # Chrome 옵션 설정
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--window-size=1920x1080")
    
    # WebDriver 초기화
    driver = webdriver.Chrome(options=chrome_options)
    wait = WebDriverWait(driver, 15)

    try:
        # 1. 웹사이트 접속
        driver.get("https://imgpresso.co.kr")
        
        # 2. 이미지 업로드
        upload_input = wait.until(EC.presence_of_element_located(
            (By.XPATH, "//input[@type='file']")))
        upload_input.send_keys("/absolute/path/to/your/image.jpg")
        print("이미지 업로드 완료")

        # 3. 옵션 선택 단계
        # 드롭다운 메뉴 선택 예시 (옵션 텍스트로 선택)
        try:
            option_dropdown = wait.until(EC.presence_of_element_located(
                (By.XPATH, "//select[@id='processing-options']")))
            Select(option_dropdown).select_by_visible_text("고화질 처리")
            print("옵션 선택 완료")
        except:
            print("옵션 선택 요소를 찾을 수 없음")
            raise

        # 4. 확인 버튼 클릭
        confirm_btn = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//button[contains(text(), '확인')]")))
        confirm_btn.click()
        print("확인 버튼 클릭 완료")
        time.sleep(2)  # 옵션 적용 대기

        # 5. 처리 시작
        process_btn = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//button[contains(text(), '처리 시작')]")))
        process_btn.click()
        print("이미지 처리 시작")

        # 6. 처리 완료 대기
        wait.until(EC.visibility_of_element_located(
            (By.XPATH, "//div[contains(@class, 'complete-status')]")))
        print("이미지 처리 완료")

        # 7. 결과물 다운로드
        download_btn = wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//a[contains(text(), '다운로드')]")))
        download_btn.click()
        print("다운로드 시작")
        time.sleep(5)

    except Exception as e:
        print(f"오류 발생: {str(e)}")
        driver.save_screenshot('error_screenshot.png')
    finally:
        driver.quit()

automate_image_processing()


In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import os
import time


# 다운로드 경로 설정
DOWNLOAD_DIR = os.path.join(os.getcwd(), "/home/primarina314/Projects/MRS3/Downloads")
# os.makedirs(DOWNLOAD_DIR, exist_ok=True)

# Chrome 옵션 설정
chrome_options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": DOWNLOAD_DIR,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
}
chrome_options.add_experimental_option("prefs", prefs)

# WebDriver 초기화
# service = ChromeService(ChromeDriverManager().install())
# driver = webdriver.Chrome(service=service, options=chrome_options)
driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 30)  # 대기 시간 30초로 확장

try:
    # 1. 홈페이지 접속
    driver.get("https://imgpresso.co.kr")
    print('1st step completed')
    
    # 2. 이미지 업로드
    upload_input = wait.until(EC.presence_of_element_located(
        (By.XPATH, "//input[@type='file']")))
    upload_input.send_keys("/home/primarina314/Projects/MRS3/sample-images-png/447x372.png")  # 실제 경로로 변경 필요

    print('2nd step completed')
    
    # 3. 옵션 선택 (예시: 라디오 버튼)
    # option_xpath = "//input[@name='processingOption' and @value='Very High']"

    processing_complete = wait.until(EC.visibility_of_element_located(
        (By.XPATH, "//div[contains(@id, 'conv-area-upload')]")))

    option_xpath = "//div[@id='b-ip-profile-2']"
    option_element = wait.until(EC.element_to_be_clickable(
        (By.XPATH, option_xpath)))
    option_element.click()
    print('3rd step completed')
    
    # 4. 확인 버튼 클릭
    # confirm_btn = wait.until(EC.element_to_be_clickable(
    #     (By.XPATH, "//button[contains(text(), '용량 줄이기')]")))
    confirm_btn = wait.until(EC.element_to_be_clickable(
        (By.XPATH, "//div[@id='b-ip-upload-ok']")))
    
    confirm_btn.click()
    print('4th step completed')
    
    # 5. 처리 완료 대기
    processing_complete = wait.until(EC.visibility_of_element_located(
        (By.XPATH, "//div[contains(@id, 'conv-area-complete')]")))
    print('5th step completed')
    
    # 6. 다운로드 버튼 클릭
    download_btn = wait.until(EC.element_to_be_clickable(
        (By.XPATH, "//div[contains(@id, 'b-ip-download')]")))
    download_btn.click()
    print('6th step completed')
    
    # 7. 다운로드 완료 확인
    start_time = time.time()
    while not any(fname.endswith('.png') for fname in os.listdir(DOWNLOAD_DIR)):
        if time.time() - start_time > 60:
            raise TimeoutError("다운로드 시간 초과")
        time.sleep(1)
    print('7th step completed')
    

finally:
    driver.quit()


1st step completed
2nd step completed
3rd step completed
4th step completed
5th step completed
6th step completed
7th step completed


TODO: 기존 MRS3 와 web compress 함께 적용하여 더욱 압축

In [6]:
"""
남은 TODO
파일 확장자 새로 제작 -> 하나의 파일로 저장, 읽기, 복원이 가능하도록
앱 또는 웹 형식으로 제작

"""

'\n남은 TODO\n파일 확장자 새로 제작 -> 하나의 파일로 저장, 읽기, 복원이 가능하도록\n앱 또는 웹 형식으로 제작\n\n'

In [33]:
import utils
utils.compress_imgpresso('/home/primarina314/Projects/MRS3/sample-images-png/500x500.png', '/home/primarina314/Projects/MRS3/Downloads/')


